In [2]:
from collections import defaultdict
import csv
import sys
import os

import cv2
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.affinity
import numpy as np
import tifffile as tiff

csv.field_size_limit(sys.maxsize)

ROOT_PATH = "/data/data_gWkkHSkq/kaggle-dstl"

IM_ID = '6120_2_2'
POLY_TYPE = '1'  # buildings

In [ ]:
# Load grid size
x_max = y_min = None
for _im_id, _x, _y in csv.reader(open(os.path.join(ROOT_PATH,'grid_sizes.csv'))):
    if _im_id == IM_ID:
        x_max, y_min = float(_x), float(_y)
        break


In [ ]:
# Load train poly with shapely
train_polygons = None
for _im_id, _poly_type, _poly in csv.reader(open(os.path.join(ROOT_PATH,'train_wkt_v4.csv'))):
    if _im_id == IM_ID and _poly_type == POLY_TYPE:
        train_polygons = shapely.wkt.loads(_poly)
        break

In [ ]:
# Read image with tiff
im_rgb = tiff.imread('/data/data_gWkkHSkq/kaggle-dstl/three_band/{}.tif'.format(IM_ID)).transpose([1, 2, 0])
im_size = im_rgb.shape[:2]


In [ ]:
#Scale polygons to match image:

def get_scalers():
    h, w = im_size  # they are flipped so that mask_for_polygons works correctly
    w_ = w * (w / (w + 1))
    h_ = h * (h / (h + 1))
    return w_ / x_max, h_ / y_min

x_scaler, y_scaler = get_scalers()
